<a href="https://colab.research.google.com/github/IshanSFernando/Real-and-Synthetic-image-classification/blob/main/Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

train = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/My Drive/version_1',
      labels = "inferred",
      label_mode = "categorical",
      color_mode = "rgb",
      batch_size = 32,
      image_size = (200,200),
      seed = 1234,
      subset = "training",
      validation_split = 0.2
)

valid = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/My Drive/version_1',
      labels = "inferred",
      label_mode = "categorical",
      color_mode = "rgb",
      batch_size = 32,
      image_size = (200,200),
      seed = 1234,
      subset = "validation",
      validation_split = 0.2
)

Found 3210 files belonging to 2 classes.
Using 2568 files for training.
Found 3210 files belonging to 2 classes.
Using 642 files for validation.


In [ ]:
classnames = train.class_names
print(classnames)#print classnames
num_classes = len(classnames)

['real', 'synthetic']
<BatchDataset element_spec=(TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>


In [ ]:
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import add
from keras.regularizers import l2
from keras import backend as K

In [ ]:
class ResNet:
  @staticmethod
  def residual_module(data, K, stride, chanDim, red=False,reg=0.0001, bnEps=2e-5, bnMom=0.9):
    shortcut = data
    bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(data)
    act1 = Activation("relu")(bn1)
    conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False,kernel_regularizer=l2(reg))(act1)
   
    bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv1)
    act2 = Activation("relu")(bn2)
    conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride, padding="same", use_bias=False,kernel_regularizer=l2(reg))(act2)
    
    bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps,momentum=bnMom)(conv2)
    act3 = Activation("relu")(bn3)
    conv3 = Conv2D(K, (1, 1), use_bias=False, kernel_regularizer=l2(reg))(act3)

    if red:
      shortcut = Conv2D(K, (1, 1), strides=stride, use_bias=False,kernel_regularizer=l2(reg))(act1)
    x = add([conv3, shortcut])
    return x

  @staticmethod
  def build(width, height, depth, classes, stages, filters,reg=0.0001, bnEps=2e-5, bnMom=0.9):  
    inputShape = (height, width, depth)
    chanDim = -1
    if K.image_data_format() == "channels_first":
      inputShape = (depth, height, width)
      chanDim = 1 

    inputs = Input(shape=inputShape)
    x = BatchNormalization(axis=chanDim, epsilon=bnEps,momentum=bnMom)(inputs)
    x = Conv2D(filters[0], (5, 5), use_bias=False,padding="same", kernel_regularizer=l2(reg))(x)
    x = BatchNormalization(axis=chanDim, epsilon=bnEps,momentum=bnMom)(x)
    x = Activation("relu")(x)
    x = ZeroPadding2D((1, 1))(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    for i in range(0, len(stages)):
      stride = (1, 1) if i == 0 else (2, 2)
      x = ResNet.residual_module(x, filters[i + 1], stride, chanDim, red=True, bnEps=bnEps, bnMom=bnMom)
      for j in range(0, stages[i] - 1):
        x = ResNet.residual_module(x, filters[i + 1],(1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)

    x = BatchNormalization(axis=chanDim, epsilon=bnEps,momentum=bnMom)(x)
    x = Activation("relu")(x)
    x = AveragePooling2D((8, 8))(x)
    x = Flatten()(x)
    x = Dense(classes, kernel_regularizer=l2(reg))(x)
    x = Activation("softmax")(x)
    
    model = Model(inputs, x, name="resnet")
    
    return model

In [ ]:
resnet = ResNet.build(200,200,3,2,[3,4,6],[64,128,256,512])

In [ ]:
resnet.summary()

Model: "resnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 batch_normalization (BatchNorm  (None, 200, 200, 3)  12         ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 conv2d (Conv2D)                (None, 200, 200, 64  4800        ['batch_normalization[0][0]']    
                                )                                                            

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(resnet, to_file='resnet.png', show_shapes=True, show_layer_names=True)

In [ ]:
%pip install -q wandb
import wandb
from wandb.keras import WandbCallback
wandb.login()

     |████████████████████████████████| 1.8 MB 8.4 MB/s 
     |████████████████████████████████| 181 kB 63.0 MB/s 
     |████████████████████████████████| 157 kB 73.9 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 157 kB 74.0 MB/s 
     |████████████████████████████████| 157 kB 64.1 MB/s 
     |████████████████████████████████| 157 kB 77.9 MB/s 
     |████████████████████████████████| 156 kB 76.3 MB/s 


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.init(project="resnet")

wandb: Currently logged in as: ishan-f. Use `wandb login --relogin` to force relogin


In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
resnet.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

#print(tf.keras.utils.plot_model(model))
hist=resnet.fit(train, validation_data = valid, epochs = 100, batch_size = 32,callbacks=[WandbCallback()])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Epoch 1/100
81/81 [==============================] - ETA: 0s - loss: 1.1234 - accuracy: 0.7146

wandb: Adding directory to artifact (/content/wandb/run-20220812_015912-o6et144k/files/model-best)... Done. 0.2s


81/81 [==============================] - 352s 4s/step - loss: 1.1234 - accuracy: 0.7146 - val_loss: 0.9492 - val_accuracy: 0.8505
Epoch 2/100
81/81 [==============================] - ETA: 0s - loss: 0.8521 - accuracy: 0.8563

wandb: Adding directory to artifact (/content/wandb/run-20220812_015912-o6et144k/files/model-best)... Done. 0.2s


...

81/81 [==============================] - 53s 648ms/step - loss: 0.1678 - accuracy: 0.9739 - val_loss: 0.2922 - val_accuracy: 0.9252
Epoch 44/100
81/81 [==============================] - 37s 450ms/step - loss: 0.1418 - accuracy: 0.9798 - val_loss: 0.3643 - val_accuracy: 0.8863
Epoch 45/100
81/81 [==============================] - 38s 458ms/step - loss: 0.1343 - accuracy: 0.9821 - val_loss: 0.4208 - val_accuracy: 0.9003
Epoch 46/100
81/81 [==============================] - 37s 447ms/step - loss: 0.1142 - accuracy: 0.9895 - val_loss: 0.3943 - val_accuracy: 0.9081
Epoch 47/100
81/81 [==============================] - 37s 448ms/step - loss: 0.1374 - accuracy: 0.9786 - val_loss: 0.3669 - val_accuracy: 0.9237
Epoch 48/100
81/81 [==============================] - 37s 448ms/step - loss: 0.2749 - accuracy: 0.9350 - val_loss: 0.2941 - val_accuracy: 0.9268
Epoch 49/100
81/81 [==============================] - 37s 448ms/step - loss: 0.1588 - accuracy: 0.9735 - val_loss: 0.3952 - val_accuracy: 0.891

wandb: Adding directory to artifact (/content/wandb/run-20220812_015912-o6et144k/files/model-best)... Done. 0.2s


81/81 [==============================] - 54s 663ms/step - loss: 0.1912 - accuracy: 0.9595 - val_loss: 0.2878 - val_accuracy: 0.9299
Epoch 54/100
81/81 [==============================] - 38s 451ms/step - loss: 0.1615 - accuracy: 0.9743 - val_loss: 0.6828 - val_accuracy: 0.8302
Epoch 55/100
81/81 [==============================] - 38s 457ms/step - loss: 0.1986 - accuracy: 0.9568 - val_loss: 0.3163 - val_accuracy: 0.9097
Epoch 56/100
81/81 [==============================] - 37s 446ms/step - loss: 0.1566 - accuracy: 0.9743 - val_loss: 0.3662 - val_accuracy: 0.9050
Epoch 57/100
81/81 [==============================] - 37s 449ms/step - loss: 0.1577 - accuracy: 0.9743 - val_loss: 0.3137 - val_accuracy: 0.9237
Epoch 58/100
81/81 [==============================] - 37s 450ms/step - loss: 0.1195 - accuracy: 0.9875 - val_loss: 0.3396 - val_accuracy: 0.9221
Epoch 59/100
81/81 [==============================] - 37s 447ms/step - loss: 0.1020 - accuracy: 0.9899 - val_loss: 0.3234 - val_accuracy: 0.920

wandb: Adding directory to artifact (/content/wandb/run-20220812_015912-o6et144k/files/model-best)... Done. 0.2s


81/81 [==============================] - 53s 642ms/step - loss: 0.1679 - accuracy: 0.9673 - val_loss: 0.2764 - val_accuracy: 0.9377


In [ ]:
import pickle
filename="/content/drive/My Drive/images/Test.pkl"
pickle.dump(resnet,open(filename,'wb'))